In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
print(sys.version)
print(f'tf version : {tf.__version__}')

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
tf version : 2.7.0


# TFRecords

references
- https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564
- 

In [4]:
train_df = pd.read_parquet("../datasets/h&m/hm_pp_train.parquet")

In [5]:
train_df.shape

(47850, 5)

In [6]:
train_df.head(3)

,transaction_year,prev_year_spend,prev_year_n_buy,age,price
0,0.0,0.0,0.0,0.533333,0.422034
1,0.0,0.0,0.0,0.500000,0.422034
3,0.0,0.0,0.0,0.366667,0.371356


In [7]:
train_df.dtypes

transaction_year    float64
prev_year_spend     float64
prev_year_n_buy     float64
age                 float64
price               float64
dtype: object

## custom

In [8]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [12]:
# whole df to single tfrecords

# adding compression
options = tf.io.TFRecordOptions(compression_type='GZIP', )

with tf.io.TFRecordWriter('../datasets/h&m/train_all_not_serialized.tfrecord', options=None) as writer:
    for row in train_df.itertuples():
        year = getattr(row, 'transaction_year')
        prev_year_spend = getattr(row, 'prev_year_spend')
        prev_year_n_buy = getattr(row, 'prev_year_n_buy')
        age = getattr(row, 'age')
        price = getattr(row, 'price')
        feature_dict = {
            'year': _float_feature(year),
            'prev_year_spend':_float_feature(prev_year_spend),
            'prev_year_n_buy':_float_feature(prev_year_n_buy),
            'age':_float_feature(age),
            'price':_float_feature(price)
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature_dict))
        writer.write(example_proto.SerializePartialToString())

parquet(388kB) -> TFRecord(6222kB)
- is this increased size expected?

In [ ]:
# single row to single tfrecords
feature_dict = {
    'year': _float_feature(year),
    'prev_year_spend':_float_feature(prev_year_spend),
    'prev_year_n_buy':_float_feature(prev_year_n_buy),
    'age':_float_feature(age),
    'price':_float_feature(price)
}
example_proto = tf.train.Example(features=tf.train.Features(feature=feature_dict))
with tf.io.TFRecordWriter('../datasets/h&m/train_0_single.tfrecord') as writer:
    writer.write(example_proto.SerializePartialToString())

In [44]:
# reading tfrecods

feature_desc = {
    "year": tf.io.FixedLenFeature([], tf.float32),
    'prev_year_spend': tf.io.FixedLenFeature([], tf.float32),
    'prev_year_n_buy': tf.io.FixedLenFeature([], tf.float32),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'price': tf.io.FixedLenFeature([], tf.float32)}


# single file
serialized_dataset = tf.data.TFRecordDataset('../datasets/h&m/train_all.tfrecord')

In [54]:
def parse(serialized_example):
    example = tf.io.parse_single_example(serialized_example, feature_desc)
    y = example.pop('price')
    return example, y
    
dataset = serialized_dataset.map(parse)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [49]:
"""
tensorflow Linear Regression

Does really bad on test set, values are way off compared to sklearn...
"""
inputs = tf.keras.layers.Input(shape=(4,))
outputs = tf.keras.layers.Dense(1, activation=None)(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss = tf.keras.losses.mean_squared_error,
    optimizer = 'sgd',
    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history = model.fit(dataset)

In [ ]:
for e in dataset.take(1):
    print(e)

## spark-tensorflow-connector

Writing tfrecords from pyspark dataframe.

In [177]:
full_path = os.path.join(os.getcwd(), 'spark-tensorflow-connector-1.0.0-s_2.11.jar')

In [185]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
            .appName('stc-test')\
            .config('spark.jars', 'spark-tensorflow-connector-1.0.0-s_2.11.jar')\
            .getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [186]:
import pyspark
print(pyspark.__version__)

3.2.0


In [187]:
train_pdf = spark.createDataFrame(train_df)

In [188]:
train_pdf.show(2)

+----------------+---------------+---------------+------------------+------------------+
|transaction_year|prev_year_spend|prev_year_n_buy|               age|             price|
+----------------+---------------+---------------+------------------+------------------+
|               0|            0.0|            0.0|0.5333333333333333|0.4220338983050847|
|               1|            0.0|            0.0|               0.5|0.4220338983050847|
+----------------+---------------+---------------+------------------+------------------+
only showing top 2 rows



In [190]:
# http://spark.apache.org/third-party-projects.html
train_pdf.write.format('tfrecords').option('writeLocality', 'local').save("/tfrecords")

## petastorm

# Petastorm

Loading straight from regular parquet

In [103]:
rel_path = '../datasets/h&m/transaction_train_sample.parquet'

In [104]:
os.path.join('file://', os.getcwd())

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance'

In [105]:
'file://' + os.getcwd()

'file://C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance'

In [106]:
os.path.join(os.getcwd(), 'parquet_files')

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance\\parquet_files'

In [107]:
os.path.join("file://", os.path.join(os.getcwd(), 'parquet_files'))

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance\\parquet_files'

In [153]:
df = pd.read_parquet("../datasets/h&m/hm_pp_train.parquet")

In [155]:
df.head()

,transaction_year,prev_year_spend,prev_year_n_buy,age,price
0,0.0,0.000000,0.00,0.533333,0.422034
1,0.0,0.000000,0.00,0.500000,0.422034
3,0.0,0.000000,0.00,0.366667,0.371356
6,0.0,0.000000,0.00,0.333333,0.337271
8,0.0,0.072412,0.25,0.677778,0.296525


In [159]:
pdf = spark.read.parquet("../datasets/h&m/hm_pp_train.parquet")

In [161]:
pdf.show(3)

+----------------+---------------+---------------+------------------+------------------+-----------------+
|transaction_year|prev_year_spend|prev_year_n_buy|               age|             price|__index_level_0__|
+----------------+---------------+---------------+------------------+------------------+-----------------+
|             0.0|            0.0|            0.0|0.5333333333333333|0.4220338983050847|                0|
|             0.0|            0.0|            0.0|               0.5|0.4220338983050847|                1|
|             0.0|            0.0|            0.0|0.3666666666666667|0.3713559322033898|                3|
+----------------+---------------+---------------+------------------+------------------+-----------------+
only showing top 3 rows



In [176]:
pdf.write.parquet('test.parquet')

In [170]:
from petastorm import make_reader, make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset

filepath = "file:///Users/haneu/Desktop/PROJECTS/Finance/parquet_files"
print(filepath)
# with make_reader(dataset_url=filepath) as reader:
#     for row in reader:
#         print(row)

file:///Users/haneu/Desktop/PROJECTS/Finance/parquet_files


In [191]:
with make_batch_reader(dataset_url_or_urls=filepath) as reader:
    dataset = make_petastorm_dataset(reader)
    dataset = dataset.shuffle(10_000).batch(512)
    for e in dataset.take(1):
        print(e)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


C:\Users\haneu\AppData\Local\Continuum\anaconda3\lib\site-packages\petastorm\fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
C:\Users\haneu\AppData\Local\Continuum\anaconda3\lib\site-packages\petastorm\etl\dataset_metadata.py:402: FutureWarning: Specifying the 'metadata_nthreads' argument is deprecated as of pyarrow 8.0.0, and the argument will be removed in a future version
  dataset = pq.ParquetDataset(path_or_paths, filesystem=fs, validate_schema=False, metadata_nthreads=10)
C:\Users\haneu\AppData\Local\Continuum\anaconda3\lib\site-packages\petastorm\etl\dataset_metadata.py:362: FutureWarning: 'ParquetDataset.common_metadata' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version.
  if not dataset.common_metadata:
C:\Users\haneu\AppData\Local\Continuum\anaconda3\lib\site-packages\petastorm\reader.py:422: FutureWarning: Specifying the 'metadata_nt

inferred_schema_view(t_dat=<tf.Tensor: shape=(1, 100000), dtype=string, numpy=
array([[b'2020-06-09', b'2019-10-12', b'2018-12-05', ..., b'2020-08-02',
        b'2019-04-06', b'2020-04-01']], dtype=object)>, customer_id=<tf.Tensor: shape=(1, 100000), dtype=string, numpy=
array([[b'b3c880da1436f617e6927b73ebbf4bcb1b14c2a3037bc7e939ecb71c8672e798',
        b'd1e21026a54135afd647ec7508e99910bccc804bbd8a964994d090dae7d8590b',
        b'e9d7b9fc3f2f2d5ef9d86dd6593085c9e1e6380a4c6af98233ff1762a29ed50b',
        ...,
        b'b637a3e7d8b0caa947aaefd609b8d84a9ee962cf0a52a51bac507ffc2bf1b741',
        b'fe13ed72e20e33993616857eeb08137f12ad96144b017c5f4bc568eb92e9dc2c',
        b'54a4b54f8b0ee58ac9f44e7e2c6b46efa3200524259366b5efad456631e38bb2']],
      dtype=object)>, article_id=<tf.Tensor: shape=(1, 100000), dtype=int64, numpy=
array([[695632099, 723353003, 657497011, ..., 768921001, 598517002,
        761308002]], dtype=int64)>, price=<tf.Tensor: shape=(1, 100000), dtype=float64, numpy=
arra

Pyspark -> parquet -> tf

# Reviews

In [ ]:
ds2 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4,10]))

print("- - - - - - - - ds2 - - - - - - - -")
print(tf.random.uniform([4,10]))
ds3 =(tf.random.uniform([4]),
      tf.random.uniform([4, 4], maxval=100, dtype=tf.int32))

print("- - - - - - - - ds3 - - - - - - - -")
print(ds3)
ds3 = tf.data.Dataset.from_tensor_slices(ds3)

# print(ds.element_spec)
print(ds2.element_spec)
print(ds3.element_spec)

inc_ds = tf.data.Dataset.range(100)
inc_ds2 = tf.data.Dataset.range(100, 200)
comb_ds = tf.data.Dataset.zip((inc_ds, inc_ds2))
batch_ds = comb_ds.batch(10)
list(iter(comb_ds))[:3]

for e in batch_ds: # this continues until all elements 
    print(e) 
    break
    

for e in batch_ds.take(2): # only take(n), n element within ds.
    print(e)
    
"""
Since each element may be different length, especially in sequential models. tf provide Dataset.padded_batch
function
"""
# dataset = tf.data.Dataset.range(100)
# dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
# dataset = dataset.padded_batch(4, padded_shapes=(None,))

# for batch in dataset.take(2):
#     print(batch.numpy())
#     print()